In [1]:
#HAM表示非垃圾邮件，SPAM表示垃圾邮件
HAM = 0
SPAM = 1
datadir = 'data/Bayes'
sources = [
    ('beck-s.tar.gz', HAM),
    ('farmer-d.tar.gz', HAM),
    ('kaminski-v.tar.gz', HAM),
    ('kitchen-l.tar.gz', HAM),
    ('lokay-m.tar.gz', HAM),
    ('williams-w3.tar.gz', HAM),
    ('BG.tar.gz', SPAM),
    ('GP.tar.gz', SPAM),
    ('SH.tar.gz', SPAM)
]

In [2]:
def extract_tar(datafile, extractdir):
    try:
        import tarfile
    except ImportError:
        raise ImportError("You do not have tarfile installed. "
                          "Try unzipping the file outside of Python.")

    tar = tarfile.open(datafile)
    tar.extractall(path=extractdir)
    tar.close()
    print("%s successfully extracted to %s" % (datafile, extractdir))

In [3]:
# for source, _ in sources:
#     datafile = '%s/%s' % (datadir, source)
#     extract_tar(datafile, datadir)

In [4]:
import os
def read_single_file(filename):
    past_header, lines = False, []
    if os.path.isfile(filename):
        f = open(filename, encoding="latin-1")
        for line in f:
            if past_header:
                lines.append(line)
            elif line == '\n':
                past_header = True
        f.close()
    content = '\n'.join(lines)
    return filename, content

In [5]:
def read_files(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            filepath = os.path.join(root, filename)
            yield read_single_file(filepath)

In [6]:
import pandas as pd

In [7]:
pd.DataFrame({
    'model': ['Normal Bayes', 'Multinomial Bayes', 'Bernoulli Bayes'],
    'class': [
        'cv2.ml.NormalBayesClassifier_create()',
        'sklearn.naive_bayes.MultinomialNB()',
        'sklearn.naive_bayes.BernoulliNB()'
    ]
})

,model,class
0,Normal Bayes,cv2.ml.NormalBayesClassifier_create()
1,Multinomial Bayes,sklearn.naive_bayes.MultinomialNB()
2,Bernoulli Bayes,sklearn.naive_bayes.BernoulliNB()


In [8]:
def build_data_frame(extractdir, classification):
    rows = []
    index = []
    for file_name, text in read_files(extractdir):
        rows.append({'text': text, 'class': classification})
        index.append(file_name)
    data_frame = pd.DataFrame(rows, index=index)
    return data_frame

In [9]:
data = pd.DataFrame({'text': [], 'class': []})
for source, classification in sources:
    extractdir = '%s/%s' % (datadir, source[: -7])
    data = data.append(build_data_frame(extractdir, classification))

In [10]:
# data

In [20]:
from sklearn import feature_extraction
counts = feature_extraction.text.CountVectorizer(ngram_range=(1, 2))
X = counts.fit_transform(data['text'].values)

In [12]:
y = data['class'].values

In [13]:
type(X)

scipy.sparse.csr.csr_matrix

In [30]:
from sklearn import model_selection as ms
X_train, X_test, y_train, y_test = ms.train_test_split(X_new, y, test_size=0.2, random_state=42)

In [15]:
import cv2
model_norm = cv2.ml.NormalBayesClassifier_create()

In [16]:
import numpy as np
X_train_small = X_train[: 1000, : 300].toarray().astype(np.float32)
y_train_small = y_train[: 1000].astype(np.float32)
#openCV无法处理稀疏矩阵，需要先转换成一个numpy数组，然而该数据过大，因而只截取一部分数据

In [ ]:
# model_norm.train(X_train_small, cv2.ml.ROW_SAMPLE, y_train_small) 该行无法使用

In [31]:
from sklearn import naive_bayes
model_naive = naive_bayes.MultinomialNB()
model_naive.fit(X_train, y_train)

MultinomialNB()

In [32]:
model_naive.score(X_train, y_train)

0.9958473355736918

In [33]:
model_naive.score(X_test, y_test)

0.9908794162826421

In [25]:
tfidt = feature_extraction.text.TfidfTransformer()
X_new = tfidt.fit_transform(X)

In [35]:
from sklearn import metrics
metrics.confusion_matrix(y_test, model_naive.predict(X_test))

array([[3746,   84],
       [  11, 6575]], dtype=int64)